In [163]:
import pandas as pd

data_proc = r"D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data\Data_Processed"
data_raw = r"D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data\Data_Raw"
# comp_series = pd.read_stata(fr"{data_proc}\complete_series.dta")
# complete_series_wmetadata_pre = pd.read_stata(r"D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data\Data_Processed\dump\complete_series.dta")
# unicef = pd.read_stata(r"D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data\Data_Raw\UNICEF_api_08_jun_2023.dta")

In [165]:
names = pd.read_excel(fr"{data_raw}\names.xlsx")
units = pd.read_excel(fr"{data_raw}\Units.xlsx")

In [178]:
units

,code,units
0,asr,NaN
1,basic_hygiene_health,% of surveyed health facilities
2,basic_hygiene_schools,% of surveyed schools
3,basic_sanitation_health,% of surveyed health facilities
4,basic_sanitation_schools,% of surveyed schools
...,...,...
68,vacMCV1,% of population
69,vacMCV2,% of population
70,vacPCV3,% of population
71,vacPOL3,% of population


In [182]:
m = names.merge(units, left_on='name_portal', right_on='code', how='left')
m = m.drop(columns=['code_y']).rename(columns={'code_x': 'code'})
m.to_excel(fr"{data_raw}\names_v2.xlsx", index=False)

In [177]:
names

,name_portal,code,source,name,description,update,download_link,stage_life,dimension,rank
0,unicef_stillbirths,FIXME,Unicef,NaN,NaN,NaN,https://data.unicef.org/resources/data_explore...,NaN,NaN,NaN
1,unicef_u5mort,CME_MRY0T4,Unicef,Under-five mortality rate,NaN,NaN,https://data.unicef.org/resources/data_explore...,NaN,NaN,NaN
2,unicef_1524mort,CME_MRY15T24,Unicef,Mortality rate (children aged 15 to 24 years),NaN,NaN,https://data.unicef.org/resources/data_explore...,NaN,NaN,NaN
3,unicef_514mort,CME_MRY5T14,Unicef,Mortality rate (children aged 5 to 14 years),NaN,NaN,https://data.unicef.org/resources/data_explore...,NaN,NaN,NaN
4,ed_mat_g23,ED_MAT_G23,Unicef,NaN,NaN,NaN,https://data.unicef.org/resources/data_explore...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
126,u5_pneu_cs,u5_pneu_cs,WHO,Children aged < 5 years with pneumonia symptom...,Percentage of children aged 0–59 months with p...,"2022, May",https://www.who.int/data/gho/data/indicators,NaN,NaN,NaN
127,fao_undernourishment,v210041,FAO,Prevalence of undernourishment (percent) (3-ye...,The prevalence of undernourishment expresses t...,"2022, July",https://www.fao.org/faostat/es/#data/FS,NaN,NaN,NaN
128,fao_wasting,v21026,FAO,Percentage of children under 5 years affected ...,Weight-for-height is more than two standard d...,"2022, July",https://www.fao.org/faostat/es/#data/FS,NaN,NaN,NaN
129,fao_stunting,v21049,FAO,Prevalence of low birthweight (percent),Low birthweight is defined as a weight at birt...,"2022, July",https://www.fao.org/faostat/es/#data/FS,NaN,NaN,NaN


In [164]:
complete_series_wmetadata = pd.read_stata(fr"{data_proc}\complete_series_wmetadata.dta")
names = pd.read_excel(fr"{data_raw}\names.xlsx")
coded_names = names.code.unique()
variables = pd.Series(complete_series_wmetadata.code.unique())
assert variables.isin(coded_names).all(), "There are variables without metadata in names"

AssertionError: Not all variables in final dataset have metadata

In [149]:
comp_series = pd.read_stata(r"D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data\Data_Processed\comp_series.dta")

In [154]:
comp_series

,wbcode,year,gender,eap_2wap_mf_a,eip_2plf_a,eip_2plf_y,eip_2wap_a,eip_2wap_y,eip_neet_mf_y,emp_nifl_a,emp_nifl_y,lastnm_afr,lastnm_birth_reg,lastnm_mmrt,lastnm_probdeath_ncd,lastnm_sec_ger,lastnm_ter_ger,luu_2lu4_mf_a,luu_2lu4_mf_y,se_lpv_prim,sp_dyn_le00_in,uiscr1,uiscr2,uiscr3,uisger01,uisger02,uisoaepg1,uisoaepg2gpv,uisqutp1,uisqutp2t3,uisschbsp1welec,uisxgdpfsgov,uisxgovexpimf,une_2eap_mf_a,une_2eap_mf_y,unicef_breastf,unicef_care,unicef_diarrhoea,unicef_mealfreq,unicef_neomort,vacbcg,vachepbb
0,AFG,2021,0.0,41.279999,NaN,NaN,54.018002,66.259003,27.799999,85.099998,90.099998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.88011,10.226,20.226000,NaN,NaN,NaN,NaN,NaN,84.0,39.0
1,AFG,2021,1.0,68.182999,NaN,NaN,24.788000,47.992001,8.900000,81.699997,87.099998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.345,18.599001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,2021,2.0,12.834000,NaN,NaN,84.922997,85.533997,44.500000,96.800003,98.599998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.876,26.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,2004,0.0,87.527000,NaN,NaN,10.583000,43.027000,31.000000,77.199997,88.900002,188.1908,NaN,NaN,NaN,NaN,1.97574,NaN,NaN,NaN,50.592,NaN,NaN,14.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.114,8.626000,NaN,NaN,NaN,NaN,45.1,63.0,NaN
4,AGO,2011,0.0,85.641998,4.911,23.039,11.021000,42.486000,10.000000,80.699997,95.199997,170.1658,NaN,NaN,NaN,26.48254,6.13030,17.271999,43.726002,NaN,57.596,46.187149,20.738819,17.1,0.0,73.328979,NaN,NaN,NaN,NaN,NaN,3.03,8.94000,3.750,16.847000,NaN,NaN,NaN,NaN,35.1,74.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39895,ZWE,2016,2.0,81.249001,8.379,14.599,15.277000,28.622999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.54473,27.976000,36.668999,NaN,62.416,99.207138,63.612911,NaN,NaN,67.464783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.101,7.784000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39896,ZWE,2017,2.0,81.231003,8.376,14.690,15.298000,28.723000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.68927,27.823000,36.493999,NaN,62.956,96.014954,65.054382,NaN,NaN,70.503242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.099,7.842000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39897,ZWE,2018,2.0,81.309998,8.360,14.797,15.208000,28.922001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.632999,36.286999,NaN,63.594,93.238541,59.555260,NaN,NaN,69.573807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.106,7.909000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39898,ZWE,2020,2.0,79.772003,10.026,16.635,16.361000,30.596001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.862,90.902550,67.054733,NaN,NaN,72.631767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.624,8.681000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
variables[-variables.isin(coded_names)]

102    unicef_stillbirths
106                vacbcg
107              vachepbb
dtype: object

In [133]:
complete_series_wmetadata[complete_series_wmetadata.code == 'emp_nifl_a']

AttributeError: 'DataFrame' object has no attribute 'code'

In [126]:
df = pd.read_stata(fr'{data_proc}\all_unicef.dta')

In [128]:
df

,year,CME_MRY0T4,CME_MRY15T24,CME_MRY5T14,ED_MAT_G23,ED_READ_G23,HVA_EPI_DTH_ANN_15_24,HVA_EPI_DTH_RT_0_14,HVA_EPI_INF_RT_0_14,HVA_EPI_INF_RT_10_19,HVA_PED_ART_CVG,HVA_PMTCT_MTCT,HVA_PREV_KNOW,HVA_PREV_KNOW_TEST,IM_BCG,IM_DTP1,IM_DTP3,IM_HEPB3,IM_HEPBB,IM_HIB3,IM_IPV1,IM_MCV1,IM_MCV2,IM_PCV3,IM_POL3,IM_ROTAC,MNCH_ANC1,MNCH_ANC4,MNCH_DEMAND_FP,MNCH_ITN,MNCH_ITNPREG,MNCH_MLRACT,MNCH_MLRCARE,MNCH_MLRDIAG,MNCH_ORS,MNCH_PNCNB,MNCH_PNEUCARE,MNCH_SAB,NT_ANT_HAZ_NE2,NT_ANT_WHZ_NE2,NT_ANT_WHZ_NE3,NT_ANT_WHZ_PO2,PT_CHLD_5_17_LBR_ECON,PT_F_18_29_SX_V_AGE_18,PT_M_18_29_SX_V_AGE_18,WS_HCF_H_B,WS_HCF_S_B,WS_HCF_W_B,WS_PPL_H_B,WS_PPL_S_ALB,WS_PPL_W_ALB,WS_SCH_H_B,WS_SCH_S_B,WS_SCH_W_B,gender,wbcode,unicef_stillbirths
0,2016,67.248306,31.468534,5.379400,NaN,NaN,100.0,0.38,0.02,0.02,9.1,46.92,NaN,NaN,78.0,73,66,66.0,11.0,66.0,66.0,64,40.0,62.0,71,NaN,NaN,NaN,42.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.767723,44.201218,64.054886,7.726976,38.2904,65.632665,0.0,AFG,28.050188
1,2017,64.581928,31.693028,4.955642,NaN,NaN,100.0,0.37,0.02,0.02,13.1,45.26,NaN,NaN,84.0,75,66,66.0,21.0,66.0,65.0,67,40.0,66.0,71,NaN,70.2,16.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.847134,45.789127,66.808701,7.726976,38.2904,65.632665,0.0,AFG,27.651554
2,2018,62.171774,31.612396,4.642200,NaN,NaN,100.0,0.37,0.02,0.02,13.5,46.12,NaN,NaN,89.0,81,72,72.0,29.0,72.0,71.0,71,49.0,69.0,76,45.0,65.2,20.9,NaN,NaN,NaN,NaN,62.1,NaN,39.9,19.3,67.7,58.8,38.2,5.1,1.6,4.1,NaN,NaN,NaN,NaN,NaN,NaN,37.931721,47.390663,69.601929,7.726976,38.2904,65.632665,0.0,AFG,27.192472
3,2019,59.917328,31.225716,4.383454,NaN,NaN,100.0,0.38,0.02,0.02,12.6,45.77,NaN,NaN,86.0,75,72,72.0,36.0,72.0,70.0,64,41.0,68.0,75,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.021141,49.006168,72.434219,7.726976,38.2904,65.632665,0.0,AFG,26.737708
4,2020,57.797811,30.684949,4.169155,NaN,NaN,100.0,0.40,0.02,0.02,12.4,44.93,NaN,NaN,87.0,78,70,70.0,38.0,70.0,65.0,66,43.0,68.0,75,62.0,NaN,27.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.8,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,38.115051,50.499645,75.091415,7.726976,38.2904,65.632665,0.0,AFG,26.187832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2017,56.175125,25.524423,12.421910,NaN,NaN,2700.0,66.21,2.48,3.49,72.3,9.10,NaN,NaN,95.0,94,89,89.0,NaN,89.0,NaN,90,78.0,89.0,89,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.200001,17.1,80.800003,42.425129,36.941673,63.996624,NaN,NaN,58.700335,0.0,ZWE,15.985734
1148,2018,53.687693,25.075499,12.085535,NaN,NaN,2700.0,50.61,2.11,2.78,59.1,8.11,NaN,NaN,95.0,94,89,89.0,NaN,89.0,NaN,88,78.0,89.0,89,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.200001,17.1,80.800003,42.419643,36.357159,63.538773,NaN,NaN,59.294202,0.0,ZWE,16.161966
1149,2019,52.665884,24.623577,11.755944,5.1,19.6,2500.0,49.50,1.89,2.30,72.0,7.70,NaN,NaN,95.0,94,90,90.0,NaN,90.0,65.0,85,75.0,90.0,90,92.0,93.3,71.5,NaN,14.9,NaN,NaN,NaN,12.2,32.8,90.9,48.0,86.0,23.5,2.9,0.3,2.5,25.6,2.4,NaN,58.200001,17.1,80.800003,42.419842,35.774338,63.094955,NaN,NaN,59.888069,0.0,ZWE,17.079875
1150,2020,51.769599,24.219571,11.433348,NaN,NaN,2300.0,50.60,1.94,2.03,72.9,8.38,NaN,NaN,88.0,93,86,86.0,NaN,86.0,86.0,85,74.0,86.0,86,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.200001,17.1,80.800003,42.426109,35.192364,62.666458,NaN,NaN,60.481936,0.0,ZWE,18.633209


In [15]:
print(complete_series_wmetadata_pre.columns.shape[0])
complete_series_wmetadata_pre.columns

111


Index(['wbcode', 'year', 'gender', 'eap_2wap_mf_a', 'eip_2plf_a', 'eip_2plf_y',
       'eip_2wap_a', 'eip_2wap_y', 'eip_neet_mf_y', 'emp_nifl_a',
       ...
       'WHS4_106', 'ECedu', 'repetition', 'v21049', 'v210041', 'v21026',
       'refugees', 'A_seekers', 'IDPs', 'population'],
      dtype='object', length=111)

In [28]:
complete_series_wmetadata.code.unique()

array(['AID_deaths_014', 'AID_deaths_1524', 'A_seekers', 'CME_MRY0T4',
       'CME_MRY15T24', 'CME_MRY5T14', 'ECedu', 'HIV_inc_014',
       'HIV_inc_1019', 'HIV_mother_child', 'HVA_PED_ART_CVG',
       'HVA_PREV_KNOW', 'HVA_PREV_KNOW_TEST', 'IDPs', 'IM_BCG', 'IM_DTP1',
       'IM_DTP3', 'IM_HEPB3', 'IM_HEPBB', 'IM_HIB3', 'IM_IPV1', 'IM_MCV1',
       'IM_MCV2', 'IM_PCV3', 'IM_POL3', 'IM_ROTAC', 'ITN_children',
       'ITN_pregnant', 'MNCH_DEMAND_FP', 'MNCH_MLRACT', 'MNCH_MLRDIAG',
       'MNCH_PNCNB', 'MNCH_PNEUCARE', 'MNCH_SAB', 'ORS_children',
       'PT_F_18_29_SX_V_AGE_18', 'PT_M_18_29_SX_V_AGE_18', 'WS_HCF_H_B',
       'WS_HCF_S_B', 'WS_HCF_W_B', 'WS_PPL_H_B', 'WS_PPL_S_ALB',
       'WS_PPL_W_ALB', 'WS_SCH_H_B', 'WS_SCH_S_B', 'WS_SCH_W_B',
       'antenatal_care_1', 'antenatal_care_4', 'asr',
       'care_febrile_children', 'child_labor', 'eap_2wap_mf_a',
       'ed_mat_g23', 'ed_read_g23', 'eip_2plf_a', 'eip_2plf_y',
       'eip_2wap_a', 'eip_2wap_y', 'eip_neet_mf_y', 'emp_nifl_a'

In [111]:
df_new = pd.read_stata(r"D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data\Data_Output\complete_series_wmd_08_jun_2023.dta")
df_new = df_new.dropna(subset=["value"])
df_old = pd.read_stata("D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data\Data_Processed\complete_series_wmetadata.dta")
df_old = df_old.dropna(subset=["value"])


In [112]:
print(df_new.shape[0], df_old.shape[0])

449408 449408


In [113]:
df_new

,wbcode,wbcountryname,wbregion,wbincome,year,code,gender,name,description,units,scale,value,update,timespan,minyear,maxyear,data,stage_life,topic,source,download_link,note,item_code
0,ABW,Aruba,Latin America & Caribbean,High income,1972,lastnm_afr,Total,"Adolescent fertility rate (births per 1,000 wo...",Adolescent fertility rate is the number of bir...,"per 1,000 women ages 15-19",0 - not bounded,59.9130,"2023, February",1972 - 2020,1972,2020,HCCI,Youth,Health,World Bank DataBank,https://databank.worldbank.org/source/world-de...,,NaN
7,ABW,Aruba,Latin America & Caribbean,High income,1972,sp_dyn_le00_in,Total,Life expectancy at birth (years),Life expectancy at birth indicates the number ...,years,0 - not bounded,68.5770,"2023, February",1972 - 2020,1972,2020,HCCI,All Ages,Health,World Bank DataBank,https://databank.worldbank.org/source/world-de...,,NaN
8,ABW,Aruba,Latin America & Caribbean,High income,1972,sp_dyn_le00_in,Male,"Life expectancy at birth (years), Male",Life expectancy at birth indicates the number ...,years,0 - not bounded,64.5160,"2023, February",1972 - 2020,1972,2020,HCCI,All Ages,Health,World Bank DataBank,https://databank.worldbank.org/source/world-de...,,NaN
9,ABW,Aruba,Latin America & Caribbean,High income,1972,sp_dyn_le00_in,Female,"Life expectancy at birth (years), Female",Life expectancy at birth indicates the number ...,years,0 - not bounded,72.7850,"2023, February",1972 - 2020,1972,2020,HCCI,All Ages,Health,World Bank DataBank,https://databank.worldbank.org/source/world-de...,,NaN
21,ABW,Aruba,Latin America & Caribbean,High income,1973,lastnm_afr,Total,"Adolescent fertility rate (births per 1,000 wo...",Adolescent fertility rate is the number of bir...,"per 1,000 women ages 15-19",0 - not bounded,59.0044,"2023, February",1972 - 2020,1972,2020,HCCI,Youth,Health,World Bank DataBank,https://databank.worldbank.org/source/world-de...,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282609,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,2021,vacbcg,Total,,,,0 - not bounded,88.0000,,1980 - 2021,1980,2021,HCCI,,,,,,NaN
1282618,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,2022,A_seekers,Total,,,,0 - not bounded,13645.0000,,2015 - 2022,2015,2022,HCCI,,,,,,NaN
1282628,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,2022,IDPs,Total,,,,0 - not bounded,0.0000,,2015 - 2022,2015,2022,HCCI,,,,,,NaN
1282674,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,2022,population,Total,,,,0 - not bounded,16.3200,,2015 - 2022,2015,2022,HCCI,,,,,,NaN
